In [1]:
import pandas as pd
import pyodbc
import os
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

from datetime import datetime
import numpy as np


print("Script Run Started")

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:/Users/Administrator/LIS/Dabur/python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

# Open Sheet
sheet = client.open_by_key("17NjdYnTDDdj8ssSLP9L06XuXlCEsQDIiIa6rHWICd00") # Master Sheets ( Dimensions Sheet)
Data = sheet.worksheet("Data")
UserSheet = sheet.worksheet("User")
Attendance = sheet.worksheet("Attendance")

server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

query = """ 
WITH CTE AS 
(SELECT 
(TXT.UserName+'-'+ TXT.VISIT_DATE)AS UserVisitKey ,
TXT.Visit_Date,
TXT.UserName,
(TXT.VISIT_DATE +' '+TXT.CHEKINTIME) AS ChekinTime,
TXT.LISStoreCode,
(TXT.VISIT_DATE+'-'+TXT.LISStoreCode+'-'+TXT.UserName) AS StoreDateKey,
TXT.StoreName,TXT.Designation,
(TXT.VISIT_DATE +' '+TXT.CHECKOUTTIME ) AS Chekouttime,
TXT.FullName,
NoPermissionReason
FROM
(SELECT   FORMAT(CONVERT(DATETIME, SA.[VisitDate], 105), 'dd-MM-yyyy') AS Visit_Date,
SA.LISStoreCode,SA.StoreName, SA.UserName,
SA.Designation,SA.FullName,
CONVERT(varchar, CAST([CheckIn-Time] AS TIME), 8) AS CHEKINTIME,
CONVERT(varchar, CAST([CheckOut-Time] AS TIME), 8) AS CHECKOUTTIME,
NoPermissionReason
FROM [PowerBI_LISV2].[GCPLFoodsManagement].[ISPStoreAttendance_NormalExport] AS SA
LEFT JOIN [PowerBI_LISV2].[GCPLFoodsManagement].UserMaster AS UM ON UM.UserName = SA.UserName
WHERE  [Month] = MONTH(GETDATE()-1) AND [Year] = YEAR(GETDATE()-1)
AND UM.Status =  'True' and  SA.UserName not like '%Star%' and SA.UserName not like '%TEST%'
)AS TXT
)
SELECT * FROM CTE;
"""

UserDataQuery ="""
WITH CTE AS
(    SELECT 
        UM2.UserMasterId,
		UM2.EmployeeID,
        UM2.UserName,
        UM2.Status,
		UM2.FullName,
		UM2.Designation,
        UM2.LastWorkingDate,
        UM2.Region,
		UM2.State,
        UM2.City,
        RANK() OVER (PARTITION BY UM2.UserName ORDER BY UM2.UserMasterId DESC) AS RK
    FROM [PowerBI_LISV2].[GCPLFoodsManagement].UserMaster AS UM2
    -- WHERE UM2.UserName IN ('GCPLF-ISP-136')
	)
SELECT CTE.EmployeeID,CTE.UserName,CTE.Status,CTE.FullName,CTE.Designation,CTE.Region,CTE.State,CTE.City,
SUP.Supervisor
FROM CTE
LEFT JOIN
	(SELECT ISP,Supervisor FROM
		(SELECT *,RANK() OVER (PARTITION BY UH.ISP ORDER BY UH.UserHierarchyListId DESC) AS RK FROM
			[PowerBI_LISV2].[GCPLFoodsManagement].UserHierarchyListReport AS UH)AS UH WHERE UH.RK =1) AS SUP
			ON SUP.ISP = CTE.UserName
WHERE CTE.RK = 1 AND CTE.Status = 'True' and 
(CTE.UserName like '%ISP%' OR CTE.UserName like '%MR%' OR CTE.UserName like '%SUP%');

""" 



query2attendance = """
WITH Attendance_CTE AS (
    SELECT 
        A.UserName,
        A.FullName,
        A.Region,
        A.Designation,
        A.PositionCode,
        CASE
            WHEN A.Status = 'Present' THEN 'P'
            WHEN A.Status = 'Absent' THEN 'A'
            WHEN A.Status = 'Leave' THEN 'L'
            WHEN A.Status = 'Compensatory Off' THEN 'CO'
            WHEN A.Status IN ('Week Off', 'WeekOff') THEN 'WO'
            WHEN A.Status = 'Meeting' THEN 'M'
            WHEN A.Status = 'Training' THEN 'T'
            WHEN A.Status IN ('National Hoilday') THEN 'H'
            WHEN A.Status = 'Holiday' THEN 'H'
            ELSE 'NA'
        END AS AttStatus,
        DAY(TRY_CONVERT(date, A.[Date], 105)) AS DayNum
    FROM PowerBI_LISV2.[GCPLFoodsManagement].ISPAttendanceMasterExport AS A
    LEFT JOIN PowerBI_LISV2.[GCPLFoodsManagement].UserMaster AS M
        ON A.UserName = M.UserName
    WHERE TRY_CONVERT(date, A.[Date], 105) >= 
    CASE 
        WHEN DAY(GETDATE()) = 1 
        THEN DATEFROMPARTS(YEAR(DATEADD(MONTH, -1, GETDATE())), MONTH(DATEADD(MONTH, -1, GETDATE())), 1)
        ELSE DATEFROMPARTS(YEAR(GETDATE()), MONTH(GETDATE()), 1)
    END
        AND TRY_CONVERT(date, A.[Date], 105) < 
    CASE 
        WHEN DAY(GETDATE()) = 1 
        THEN DATEADD(MONTH, 1, DATEFROMPARTS(YEAR(DATEADD(MONTH, -1, GETDATE())), MONTH(DATEADD(MONTH, -1, GETDATE())), 1))
        ELSE DATEADD(MONTH, 1, DATEFROMPARTS(YEAR(GETDATE()), MONTH(GETDATE()), 1))
    END


        AND M.Status = 'True'
        AND (A.UserName LIKE '%ISP%' OR A.UserName LIKE '%MR%')
),
Pivoted AS (
    SELECT *
    FROM Attendance_CTE
    PIVOT (
        MAX(AttStatus)
        FOR DayNum IN (
            [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
            [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
            [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31]
        )
    ) AS P
)
SELECT *
FROM Pivoted
ORDER BY UserName;

    
"""



conn = pyodbc.connect(conn_str)
df = pd.read_sql(query, conn)
df_attendance = pd.read_sql(query2attendance, conn)

UserData = pd.read_sql(UserDataQuery, conn)
Supervisor = UserData[UserData['Designation']=='Supervisor']
Supervisor2 = Supervisor[['UserName','FullName']]
Supervisor2 = Supervisor2.rename(columns={'UserName':'Supervisor','FullName':'SupervisorName'})
Supervisor2
dfUser = pd.merge(UserData,Supervisor2,left_on = 'Supervisor', right_on= 'Supervisor', how='left')

UserData2=dfUser.drop(columns = ['FullName','Designation'])

print("data extracted....")
print("data extracted....")
loginlogout = pd.merge(df,UserData2, how = 'left',on = 'UserName')
loginlogout['ChekinRank'] = loginlogout.groupby(['StoreDateKey'])['ChekinTime'].rank(method='dense', ascending=True).astype(int)
loginlogout2= loginlogout[loginlogout['ChekinRank']==1]

loginlogout2=loginlogout2.drop(columns = ['ChekinRank'])

loginlogout2 = loginlogout2.drop_duplicates()
loginlogout2.at[0, 'LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

print("csv updated....")
loginlogout2.to_csv(r'C:\Users\Administrator\LIS\Dabur\gcfoodsloginlogout.csv', index=False)

loginlogout2 = loginlogout2.drop_duplicates()
loginlogout2= loginlogout2[['UserVisitKey','Visit_Date','UserName','ChekinTime','LISStoreCode','StoreDateKey','StoreName','Designation',
                           'Chekouttime','FullName','NoPermissionReason','EmployeeID',
                           'Region','State','City','Status','Supervisor','SupervisorName','LastUpdated']]
#Sending df on sheet
Data.batch_clear(["A1"])
set_with_dataframe(Data, loginlogout2)

UserSheet.batch_clear(["A1"])
set_with_dataframe(UserSheet, dfUser)

Attendance.batch_clear(["A1"])
set_with_dataframe(Attendance, df_attendance)

print("Google Sheets are updated")

# emailer

# from datetime import datetime, timedelta
# import smtplib
# from email.message import EmailMessage

# sender_email   = "reports.godrejfood@loveinstore.com"
# app_password   = "sfrn ctod pwue fmxo"
# recipients     = ["swapnils@loveinstore.com","hariomc@loveinstore.com","pankajs@loveinstore.com","sumitah7899@gmail.com","delhi4jeet@gmail.com","reports.mis@loveinstore.com","amitr@loveinstore.com","raviidonneous7@gmail.com"
# ]
# subject_tpl    = "GCPLFoods_ MR Wise Daily Login & Logout Reports updated till {date}"
# body_template  = """
# Dear All,

# Please find attached the MR Wise Daily Login-Logout Reports.
# These reports provide a detailed Login and Logout times of store visits by MR & Supervisors capturing Time utilization insights for FOS and CFT activities

# Report Link :
# https://docs.google.com/spreadsheets/d/17NjdYnTDDdj8ssSLP9L06XuXlCEsQDIiIa6rHWICd00/edit?gid=1040818418#gid=1040818418

# This data will help you analyze field operations more effectively and monitor time spent at the store level.

# Note: If there are any changes or clarifications required, please let me know.

# Regards,
# Team Love In Store
# """

# now = datetime.now() - timedelta(days=1)

# rounded = now.replace(minute=0, second=0, microsecond=0)
# time_str = rounded.strftime("%I:%M %p")
# formatted_date = now.strftime("%d-%b-%Y")

# msg = EmailMessage()
# msg["From"]    = sender_email
# msg["To"]      = ", ".join(recipients)
# msg["Subject"] = subject_tpl.format(date=formatted_date, time=time_str)
# msg.set_content(body_template.format(date=formatted_date, time=time_str))

# try:
#     with smtplib.SMTP("smtp.gmail.com", 587) as smtp:
#         smtp.starttls()
#         smtp.login(sender_email, app_password)
#         smtp.send_message(msg)
#     print("✅ Email sent successfully to:", ", ".join(recipients))

#     os.remove(file_path)
#     print("✅ Deleted temporary file:", file_path)

# except Exception as e:
#     print("❌ Failed to send email:", e)


# print("done")



















Script Run Started


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_5996\982338679.py:164: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_5996\982338679.py:165: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_attendance = pd.read_sql(query2attendance, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_5996\982338679.py:167: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  UserData = pd.read_sql(UserDataQuery, conn)


data extracted....
data extracted....
csv updated....
Google Sheets are updated
